In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os

# Change Directory to file location

In [2]:
os.chdir(r'C:\Users\kjsdr\Documents\GitHub\foster-app\sample-data\Sadhanand')

# Read file content into Pandas DataFrame

In [3]:
dataset=pd.read_excel('mavoix_ml_sample_dataset.xlsx')

# Have a look at the Data

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 23 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  378 non-null object
Degree                        380 non-null object
Stream                        374 non-null object
Current 

# Looks like there are some null values for some of the columns.Need to Check whether all null values have been captured(those marked as N/A in excel file as well).check row for application ID ML16

In [5]:
id_filt=(dataset['Application_ID']=='ML0016')
dataset.loc[id_filt,['Performance_UG','Performance_12']]

,Performance_UG,Performance_12
15,8.00/10,NaN


# check empty space capture.check row for ML0025

In [6]:
id_filt=(dataset['Application_ID']=='ML0025')
dataset.loc[id_filt,['Stream']]

,Stream
24,NaN


# check percentage of NaN values for each column

In [7]:
dataset.isna().mean().round(4) * 100

Application_ID                 0.00
Current City                   0.00
Python (out of 3)              0.00
R Programming (out of 3)       0.00
Deep Learning (out of 3)       0.00
PHP (out of 3)                 0.00
MySQL (out of 3)               0.00
HTML (out of 3)                0.00
CSS (out of 3)                 0.00
JavaScript (out of 3)          0.00
AJAX (out of 3)                0.00
Bootstrap (out of 3)           0.00
MongoDB (out of 3)             0.00
Node.js (out of 3)             0.00
ReactJS (out of 3)             0.00
Other skills                   3.57
Degree                         3.06
Stream                         4.59
Current Year Of Graduation     0.00
Performance_PG                73.72
Performance_UG                13.78
Performance_12                41.58
Performance_10                45.15
dtype: float64

# even though there is a high percentage of missing values in Performance_ metrics,we are not going to remove those columns,since we deem it important for judging a candidate.

# going to impute missing values with model based imputation since it aint appropriate to take the mean/median here since large part of the columns have missing values.instead,imputation is done by taking those rows that have values for columns as training set,and predicting the values for missing data.

In [8]:
# Performance UG
from sklearn.impute import KNNImputer

# Since KNN requires other values to be numerical,we will do some feature engineering to convert columns into definable metrics and then do knn imputation.

# Degree and stream are categorical variables.lets us find the set of unique values first

In [9]:
dataset.Degree.unique()

array(['Bachelor of Science (B.Sc)', 'Bachelor of Technology (B.Tech)',
       'Master of Science (M.Sc)', 'Bachelor of Engineering (B.E)', nan,
       'PG Diploma in Data Science', 'Post Graduate Programme (PGP)',
       'Post Graduate Diploma', 'Master of Computer Applications (MCA)',
       'Master of Technology (M.Tech)', 'Executive MBA', 'MBA',
       'B.Tech (Hons.)',
       'Post Graduate Diploma in Big Data Analytics (PG-DBDA)',
       'Bachelor of Commerce (B.Com)',
       'Bachelor of Engineering (B.E) (Hons.)',
       'Bachelor of Computer Applications (BCA)', 'Integrated M.Sc.',
       'Bachelor of Business Management (B.B.M.)',
       'Bachelor of Commerce (B.Com) (Hons.)',
       'Post Graduate Diploma in Management (P.G.D.M.)',
       'Integrated B.Tech & M.Tech', 'Master of Science (M.S.)',
       'Post Graduate Program in Business Analytics',
       'Bachelor of Computer Science (B.C.S.)', 'Integrated B.Tech',
       'Master of Science (M.Sc) (Hons.)', 'Integrated M.Te

# there are too many values .high cardinality.need to club some of the values=>Bachelor program,master program,Integrated program,Post Graduate Program

In [12]:
from sklearn.preprocessing import LabelEncoder
lb_Degree = LabelEncoder()

In [14]:
dataset["Degree_code"] = lb_Degree.fit_transform(list(dataset["Degree"]))


In [15]:
dataset[["Degree", "Degree_code"]].head(11)

,Degree,Degree_code
0,Bachelor of Science (B.Sc),11
1,Bachelor of Technology (B.Tech),12
2,Master of Science (M.Sc),22
3,Bachelor of Engineering (B.E),9
4,Bachelor of Technology (B.Tech),12
5,NaN,31
6,NaN,31
7,PG Diploma in Data Science,25
8,Post Graduate Programme (PGP),30
9,Post Graduate Programme (PGP),30


In [19]:
array_degree_code=dataset.Degree_code.unique()
array_degree_code

array([11, 12, 22,  9, 31, 25, 30, 26, 20, 24, 13, 19,  1, 27,  5, 10,  7,
       17,  4,  6, 28, 16, 21, 29,  8, 15, 23, 18,  0,  3,  2, 14],
      dtype=int64)

In [20]:
array_degree=dataset.Degree.unique()
array_degree

array(['Bachelor of Science (B.Sc)', 'Bachelor of Technology (B.Tech)',
       'Master of Science (M.Sc)', 'Bachelor of Engineering (B.E)', nan,
       'PG Diploma in Data Science', 'Post Graduate Programme (PGP)',
       'Post Graduate Diploma', 'Master of Computer Applications (MCA)',
       'Master of Technology (M.Tech)', 'Executive MBA', 'MBA',
       'B.Tech (Hons.)',
       'Post Graduate Diploma in Big Data Analytics (PG-DBDA)',
       'Bachelor of Commerce (B.Com)',
       'Bachelor of Engineering (B.E) (Hons.)',
       'Bachelor of Computer Applications (BCA)', 'Integrated M.Sc.',
       'Bachelor of Business Management (B.B.M.)',
       'Bachelor of Commerce (B.Com) (Hons.)',
       'Post Graduate Diploma in Management (P.G.D.M.)',
       'Integrated B.Tech & M.Tech', 'Master of Science (M.S.)',
       'Post Graduate Program in Business Analytics',
       'Bachelor of Computer Science (B.C.S.)', 'Integrated B.Tech',
       'Master of Science (M.Sc) (Hons.)', 'Integrated M.Te

In [21]:
s1 = pd.Series(list(array_degree_code), name='degree_code')
s2 = pd.Series(list(array_degree), name='degree')
df_unique_combinations=pd.concat([s1, s2], axis=1)

In [22]:
df_unique_combinations

,degree_code,degree
0,11,Bachelor of Science (B.Sc)
1,12,Bachelor of Technology (B.Tech)
2,22,Master of Science (M.Sc)
3,9,Bachelor of Engineering (B.E)
4,31,NaN
5,25,PG Diploma in Data Science
6,30,Post Graduate Programme (PGP)
7,26,Post Graduate Diploma
8,20,Master of Computer Applications (MCA)
9,24,Master of Technology (M.Tech)


# Need to figure out which codes to combine to reduce the cardinality

In [26]:
#one set 
set1_codes=[11,12,9,1,5,10,7,4,6,8,2,3,0]#bachelor_programs
set2_codes=[22,20,24,17,21,15,23,18,14,16]#master_progrmas&integrated
set3_codes=[25,30,26,13,19,27,28,29]#post_graduate_mba program

In [27]:
len(set1_codes)+len(set2_codes)+len(set3_codes)

31

# combine codes

# update rows for bachelor_programs

In [41]:
# for each row,check whether degree_code is in set1_codes
list_flags=[False for i in range(len(dataset["Degree_code"]))]
#print(len(list_flags))     

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [46]:
list_code=list(dataset["Degree_code"].values)
#print(len(list_code))

392


In [47]:
list_index=list(range(len(dataset["Degree_code"].values)))
#print(len(list_index))

392


In [53]:
for (code,index) in zip(list_code,list_index):
    #print(code,index)
    if(code in set1_codes ):
        list_flags[index]=True 
print("no of rows belonging to set1 is :",sum(list_flags))

no of rows belonging to set1 is : 243


# check 1 corresponding row and check degree

In [56]:
len(list_flags)

392

In [57]:
len(dataset)

392

In [58]:
series_flags=pd.Series(list_flags, name='flags')
series_flags

0       True
1       True
2      False
3       True
4       True
       ...  
387     True
388    False
389     True
390     True
391    False
Name: flags, Length: 392, dtype: bool

# getting subset dataframe that belongs to set1-bachelor programs

In [65]:
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
0,ML0001,Bangalore,0,2,0,2,0,2,3,2,...,0,R Programming,Bachelor of Science (B.Sc),Mathematics,2017,NaN,NaN,NaN,NaN,bachelors program
1,ML0002,Bangalore,2,0,0,2,2,2,2,2,...,0,"Data Science, Machine Learning, Neural Network...",Bachelor of Technology (B.Tech),Computer Science & Engineering,2019,NaN,85.50/100,92.20/92.20,96.60/96.60,bachelors program
3,ML0004,Bangalore,2,0,2,1,0,2,0,0,...,0,"CSS, Deep Learning, Embedded Systems, HTML, Ma...",Bachelor of Engineering (B.E),Electronics and Communication,2019,NaN,6.86/10,76.00/76.00,84.32/84.32,bachelors program
4,ML0005,Bangalore,2,0,0,2,0,2,1,1,...,2,"HTML, OpenCV, Python, SQL, C++ Programming, CS...",Bachelor of Technology (B.Tech),Production Engineering,2018,NaN,6.38/10,65.20/65.20,68.80/68.80,bachelors program
10,ML0011,Bangalore,2,0,0,0,0,0,0,0,...,0,"CSS, HTML, Java, Python, SQL",Bachelor of Technology (B.Tech),Electronics and Communication,2018,NaN,64.00/100,95.00/95.00,NaN,bachelors program


In [62]:
dataset.loc[series_flags,'Degree_code']='bachelors program'

In [64]:
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
0,ML0001,Bangalore,0,2,0,2,0,2,3,2,...,0,R Programming,Bachelor of Science (B.Sc),Mathematics,2017,NaN,NaN,NaN,NaN,bachelors program
1,ML0002,Bangalore,2,0,0,2,2,2,2,2,...,0,"Data Science, Machine Learning, Neural Network...",Bachelor of Technology (B.Tech),Computer Science & Engineering,2019,NaN,85.50/100,92.20/92.20,96.60/96.60,bachelors program
3,ML0004,Bangalore,2,0,2,1,0,2,0,0,...,0,"CSS, Deep Learning, Embedded Systems, HTML, Ma...",Bachelor of Engineering (B.E),Electronics and Communication,2019,NaN,6.86/10,76.00/76.00,84.32/84.32,bachelors program
4,ML0005,Bangalore,2,0,0,2,0,2,1,1,...,2,"HTML, OpenCV, Python, SQL, C++ Programming, CS...",Bachelor of Technology (B.Tech),Production Engineering,2018,NaN,6.38/10,65.20/65.20,68.80/68.80,bachelors program
10,ML0011,Bangalore,2,0,0,0,0,0,0,0,...,0,"CSS, HTML, Java, Python, SQL",Bachelor of Technology (B.Tech),Electronics and Communication,2018,NaN,64.00/100,95.00/95.00,NaN,bachelors program


# update rows for master_integrated programs


In [67]:
list_flags=[False for i in range(len(dataset["Degree_code"]))]
list_code=list(dataset["Degree_code"].values)
list_index=list(range(len(dataset["Degree_code"].values)))
for (code,index) in zip(list_code,list_index):
    #print(code,index)
    if(code in set2_codes ):
        list_flags[index]=True 
print("no of rows belonging to set2 is :",sum(list_flags))

no of rows belonging to set2 is : 116


In [69]:
series_flags=pd.Series(list_flags, name='flags')

# getting subset dataframe that belongs to set2-master_integrated programs

In [70]:
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
2,ML0003,Bangalore,3,0,1,2,2,2,0,2,...,0,"Algorithms, Data Structures, Python, C Program...",Master of Science (M.Sc),Computer Science,2018,7.91/10,70.00/100,64.83/64.83,77.60/77.60,22
16,ML0017,Kozhikode,2,0,0,2,0,2,2,0,...,0,"MS-Office, Python, Data Science, Machine Learn...",Master of Science (M.Sc),Mathematics,2019,79.00/100,81.25/100,NaN,NaN,22
20,ML0021,Thrissur,1,0,0,2,2,3,3,2,...,0,"HTML, MS-Excel, SQL, Android, JavaScript, PHP,...",Master of Computer Applications (MCA),Computer Application,2017,NaN,NaN,NaN,NaN,20
21,ML0022,Bangalore,1,1,0,0,0,0,0,0,...,0,"Python, R Programming",Master of Science (M.Sc),Statistics,2019,3.76/4,3.56/4,80.50/80.50,85.85/85.85,22
24,ML0025,Lucknow,2,0,0,0,2,0,2,2,...,0,"Algorithms, C++ Programming, Data Structures, ...",Master of Computer Applications (MCA),NaN,2020,9.00/10,6.00/10,NaN,NaN,20


In [72]:
dataset.loc[series_flags,'Degree_code']='master_integrated program'
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
2,ML0003,Bangalore,3,0,1,2,2,2,0,2,...,0,"Algorithms, Data Structures, Python, C Program...",Master of Science (M.Sc),Computer Science,2018,7.91/10,70.00/100,64.83/64.83,77.60/77.60,master_integrated program
16,ML0017,Kozhikode,2,0,0,2,0,2,2,0,...,0,"MS-Office, Python, Data Science, Machine Learn...",Master of Science (M.Sc),Mathematics,2019,79.00/100,81.25/100,NaN,NaN,master_integrated program
20,ML0021,Thrissur,1,0,0,2,2,3,3,2,...,0,"HTML, MS-Excel, SQL, Android, JavaScript, PHP,...",Master of Computer Applications (MCA),Computer Application,2017,NaN,NaN,NaN,NaN,master_integrated program
21,ML0022,Bangalore,1,1,0,0,0,0,0,0,...,0,"Python, R Programming",Master of Science (M.Sc),Statistics,2019,3.76/4,3.56/4,80.50/80.50,85.85/85.85,master_integrated program
24,ML0025,Lucknow,2,0,0,0,2,0,2,2,...,0,"Algorithms, C++ Programming, Data Structures, ...",Master of Computer Applications (MCA),NaN,2020,9.00/10,6.00/10,NaN,NaN,master_integrated program


# update rows for post_graduate_mba program

In [73]:
list_flags=[False for i in range(len(dataset["Degree_code"]))]
list_code=list(dataset["Degree_code"].values)
list_index=list(range(len(dataset["Degree_code"].values)))
for (code,index) in zip(list_code,list_index):
    #print(code,index)
    if(code in set3_codes ):
        list_flags[index]=True 
print("no of rows belonging to set3 is :",sum(list_flags))

no of rows belonging to set3 is : 21


# getting subset dataframe that belongs to set3-post_graduate_mba programs

In [74]:
series_flags=pd.Series(list_flags, name='flags')
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
7,ML0008,Mumbai,3,1,0,0,0,2,0,0,...,0,"Data Analytics, Python, SQL, .NET, ASP.NET, C#...",PG Diploma in Data Science,NaN,2019,3.61/4,2.68/4,NaN,NaN,25
8,ML0009,Bangalore,3,0,0,0,0,2,2,0,...,0,"English Proficiency (Spoken), English Proficie...",Post Graduate Programme (PGP),Data Science,2020,8.00/10,7.00/10,70.00/70.00,93.00/93.00,30
9,ML0010,Bangalore,2,0,2,0,0,2,2,2,...,2,"MS-Office, Computer Vision, Data Analytics, De...",Post Graduate Programme (PGP),Artificial Intelligence And Machine Learning,2020,71.00/100,60.00/100,71.00/71.00,NaN,30
15,ML0016,Bangalore,2,0,0,0,2,2,2,1,...,0,"SQL, Data Analytics, MS-Excel, Machine Learnin...",Post Graduate Programme (PGP),Data Science And Engineering,2020,NaN,8.00/10,NaN,NaN,30
19,ML0020,Chennai,2,0,0,2,2,2,2,2,...,1,"BIG DATA ANALYTICS, Data Analytics, Database M...",Post Graduate Diploma,Data Science,2019,5.60/10,65.00/100,NaN,89.00/89.00,26


In [75]:
dataset.loc[series_flags,'Degree_code']='post_graduate_mba program'
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
7,ML0008,Mumbai,3,1,0,0,0,2,0,0,...,0,"Data Analytics, Python, SQL, .NET, ASP.NET, C#...",PG Diploma in Data Science,NaN,2019,3.61/4,2.68/4,NaN,NaN,post_graduate_mba program
8,ML0009,Bangalore,3,0,0,0,0,2,2,0,...,0,"English Proficiency (Spoken), English Proficie...",Post Graduate Programme (PGP),Data Science,2020,8.00/10,7.00/10,70.00/70.00,93.00/93.00,post_graduate_mba program
9,ML0010,Bangalore,2,0,2,0,0,2,2,2,...,2,"MS-Office, Computer Vision, Data Analytics, De...",Post Graduate Programme (PGP),Artificial Intelligence And Machine Learning,2020,71.00/100,60.00/100,71.00/71.00,NaN,post_graduate_mba program
15,ML0016,Bangalore,2,0,0,0,2,2,2,1,...,0,"SQL, Data Analytics, MS-Excel, Machine Learnin...",Post Graduate Programme (PGP),Data Science And Engineering,2020,NaN,8.00/10,NaN,NaN,post_graduate_mba program
19,ML0020,Chennai,2,0,0,2,2,2,2,2,...,1,"BIG DATA ANALYTICS, Data Analytics, Database M...",Post Graduate Diploma,Data Science,2019,5.60/10,65.00/100,NaN,89.00/89.00,post_graduate_mba program


In [76]:
dataset['Degree_code'].describe()

count                   392
unique                    4
top       bachelors program
freq                    243
Name: Degree_code, dtype: object

In [78]:
dataset.groupby(['Degree_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
Degree_code,,,,,,,,,,,,,,,,,,,,,
31,12,12,12,12,12,12,12,12,12,12,...,12,12,12,0,12,12,3,12,6,6
bachelors program,243,243,243,243,243,243,243,243,243,243,...,243,243,232,243,234,243,4,211,147,137
master_integrated program,116,116,116,116,116,116,116,116,116,116,...,116,116,113,116,108,116,84,97,65,62
post_graduate_mba program,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,20,21,12,18,11,10


# combine both master_integrated program and post_graduate_mba program into one category,since they are very similar

In [93]:
filt=(dataset['Degree_code']=='post_graduate_mba program')
dataset.loc[filt,'Degree_code'] = 'post_graduate_program'


In [94]:
dataset.groupby(['Degree_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
Degree_code,,,,,,,,,,,,,,,,,,,,,
bachelors program,255,255,255,255,255,255,255,255,255,255,...,255,255,244,243,246,255,7,223,153,143
master_integrated program,116,116,116,116,116,116,116,116,116,116,...,116,116,113,116,108,116,84,97,65,62
post_graduate_program,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,20,21,12,18,11,10


In [95]:
filt=(dataset['Degree_code']=='master_integrated program')
dataset.loc[filt,'Degree_code'] = 'post_graduate_program'
dataset.groupby(['Degree_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
Degree_code,,,,,,,,,,,,,,,,,,,,,
bachelors program,255,255,255,255,255,255,255,255,255,255,...,255,255,244,243,246,255,7,223,153,143
post_graduate_program,137,137,137,137,137,137,137,137,137,137,...,137,137,134,137,128,137,96,115,76,72


# mode imputation for degree_code=31 ,representing NaN values.

In [84]:
filt_na_flag=dataset['Degree'].isna()

In [83]:
type(dataset['Degree'].isna())

pandas.core.series.Series

In [86]:
dataset.loc[filt_na_flag,['Degree','Degree_code']].head(10)

,Degree,Degree_code
5,NaN,31
6,NaN,31
12,NaN,31
52,NaN,31
64,NaN,31
101,NaN,31
117,NaN,31
137,NaN,31
192,NaN,31
212,NaN,31


In [89]:
import statistics as st
st.mode(dataset.Degree_code)

'bachelors program'

In [90]:
dataset.loc[filt_na_flag,['Degree_code']]=st.mode(dataset.Degree_code)

In [91]:
dataset.groupby(['Degree_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
Degree_code,,,,,,,,,,,,,,,,,,,,,
bachelors program,255,255,255,255,255,255,255,255,255,255,...,255,255,244,243,246,255,7,223,153,143
master_integrated program,116,116,116,116,116,116,116,116,116,116,...,116,116,113,116,108,116,84,97,65,62
post_graduate_mba program,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,20,21,12,18,11,10


In [96]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 24 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  378 non-null object
Degree                        380 non-null object
Stream                        374 non-null object
Current 

# See unique values for Stream and do similar things to what had been done to Degree

In [97]:
dataset.Stream.unique()

array(['Mathematics', 'Computer Science & Engineering',
       'Computer  Science', 'Electronics and Communication',
       'Production Engineering', 'Data Science Engineering',
       'Artificial Intelligence', nan, 'Data Science',
       'Artificial Intelligence And Machine Learning',
       'Advanced Pg Diploma In Artificial Intelligence',
       'Data Science And Engineering', 'Computer Application',
       'Statistics', 'Electronics and Instrumentation',
       'Digital Communication & Networking',
       'Electrical and Electronics Engineering', 'Business Analytics',
       'Mechanical Engineering', 'Computer Technology',
       'Information Systems', 'Information Technology',
       'CSE With Cloud Computing', 'Data Analytics', 'Science',
       'Biological sciences and bioengineering', 'Big Data Analytics',
       'Biotechnology', 'Mechatronics', 'computer science',
       'Instrumentation & Electronics', 'Civil Engineering',
       'Industrial and Management Engineering',
    

# there are too many values .high cardinality.need to club some of the values=>advantageous_set,normal_set
# advantageous set= anything that conatins math,analytics,stat,computer,data science,artificial intelligence,software,information
# normal_set=others

In [98]:
dataset["Stream_code"] = lb_Degree.fit_transform(list(dataset["Stream"]))
dataset[["Stream", "Stream_code"]].head(11)

,Stream,Stream_code
0,Mathematics,62
1,Computer Science & Engineering,22
2,Computer Science,17
3,Electronics and Communication,42
4,Production Engineering,67
5,Data Science Engineering,33
6,Artificial Intelligence,4
7,NaN,78
8,Data Science,31
9,Artificial Intelligence And Machine Learning,5


In [99]:
array_Stream_code=dataset.Stream_code.unique()
array_Stream=dataset.Stream.unique()
s1 = pd.Series(list(array_Stream_code), name='Stream_code')
s2 = pd.Series(list(array_Stream), name='Stream')
df_unique_combinations=pd.concat([s1, s2], axis=1)
df_unique_combinations

,Stream_code,Stream
0,62,Mathematics
1,22,Computer Science & Engineering
2,17,Computer Science
3,42,Electronics and Communication
4,67,Production Engineering
...,...,...
74,53,Information Science & Engineering
75,72,Software Engineering
76,21,Computer Network And Engg
77,71,Software Systems


# There is some data available as kannur university,which is irrelevant data with respect to stream.we have to treat it as null and impute with mode after binning.

# checking for university in stream

In [103]:
dataset["Stream"].str.contains("University").sum()

1

# only 1 wrong data.that will  be handled.Stream_code 58 treated as null.

# Need to figure out which codes to combine to reduce the cardinality

# Set1=advantageous,get row index of df_unique_combinations that have rows with stream name pattern

# first mark Stream as not defined for those na values

In [107]:
dataset["Stream"].isna().sum()

18

In [108]:
filt=dataset["Stream"].isna()

In [109]:
dataset.loc[filt,['Stream']]='Not defined'

In [110]:
dataset.loc[filt,['Stream']]

,Stream
7,Not defined
24,Not defined
68,Not defined
72,Not defined
99,Not defined
107,Not defined
128,Not defined
151,Not defined
204,Not defined
210,Not defined


In [111]:
filt=dataset["Stream"].str.contains("omputer")

In [115]:
list_omputer=dataset.index[filt].tolist()


In [116]:
adv_list=[]
adv_list.extend(list_omputer)

# 167 computer related streams

In [117]:
len(adv_list)

167

In [119]:
filt=dataset["Stream"].str.contains("Stat")
list_stat=dataset.index[filt].tolist()
adv_list.extend(list_stat)
len(adv_list)

170

In [121]:
filt=dataset["Stream"].str.contains("Data")
list_data=dataset.index[filt].tolist()
adv_list.extend(list_data)
len(adv_list)

193

In [122]:
filt=dataset["Stream"].str.contains("rtificial")
list_rtificial=dataset.index[filt].tolist()
adv_list.extend(list_rtificial)
len(adv_list)

200

In [123]:
filt=dataset["Stream"].str.contains("oftware")
list_oftware=dataset.index[filt].tolist()
adv_list.extend(list_oftware)
len(adv_list)

203

In [124]:
filt=dataset["Stream"].str.contains("nformation")
list_nformation=dataset.index[filt].tolist()
adv_list.extend(list_nformation)
len(adv_list)

231

In [125]:
filt=dataset["Stream"].str.contains("nalytic")
list_nalytic=dataset.index[filt].tolist()
adv_list.extend(list_nalytic)
len(adv_list)

247

In [127]:
filt=dataset["Stream"].str.contains("Math")
list_MATH=dataset.index[filt].tolist()
adv_list.extend(list_MATH)
len(adv_list)

255

# perform sanity check

In [129]:
dataset.loc[adv_list,'Stream'].head(5)

1     Computer Science & Engineering
2                  Computer  Science
13                 Computer  Science
14    Computer Science & Engineering
18    Computer Science & Engineering
Name: Stream, dtype: object

# check whether null values are there and assign stream code to 58

In [130]:
filt=dataset["Stream"].isna()
dataset.loc[filt,['Stream_code']]='58'

In [132]:
dataset.groupby(['Stream_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
Stream_code,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
3,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,0,2,2,0,2
4,3,3,3,3,3,3,3,3,3,3,...,3,3,1,3,3,0,3,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,2,2,2,3
75,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0,0,1
76,11,11,11,11,11,11,11,11,11,11,...,11,9,11,11,11,4,7,4,2,11


# assign to Stream_code  to 2 if belonging to advantageous list,else 1

In [133]:
dataset.loc[adv_list,'Stream']

1        Computer Science & Engineering
2                     Computer  Science
13                    Computer  Science
14       Computer Science & Engineering
18       Computer Science & Engineering
                     ...               
111                         Mathematics
121                 Commerce With Maths
149                         Mathematics
213                         Mathematics
218    Computer Science And Mathematics
Name: Stream, Length: 255, dtype: object

In [134]:
dataset.loc[adv_list,'Stream_code']=2

In [145]:
filt= (dataset.Stream_code != 2) & (dataset.Stream_code !=58)

In [146]:
dataset.loc[filt,'Stream_code']

3      42
4      67
7      78
10     42
11     42
       ..
379    65
380    42
382    50
385    78
389    45
Name: Stream_code, Length: 148, dtype: int64

In [147]:
dataset.loc[filt,'Stream_code']=1

# impute not defined with mode

In [141]:
filt=(dataset.Stream_code ==58)

In [142]:
st.mode(dataset.Stream_code)

2

In [143]:
dataset.loc[filt,['Stream_code']]=st.mode(dataset.Stream_code)

In [148]:
dataset.groupby(['Stream_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
Stream_code,,,,,,,,,,,,,,,,,,,,,
1,148,148,148,148,148,148,148,148,148,148,...,148,139,146,148,148,23,129,88,85,148
2,244,244,244,244,244,244,244,244,244,244,...,244,239,234,244,244,80,209,141,130,244


# checking_data once again

In [149]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 25 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  378 non-null object
Degree                        380 non-null object
Stream                        392 non-null object
Current 

# need to work on other skills column

# identify na values

In [170]:
dataset['Other skills'].isna().sum()

14

# set na to not defined

In [172]:
filt=dataset['Other skills'].isna()
dataset.loc[filt,['Other skills']]='Not defined'

In [173]:
l=dataset['Other skills'].tolist()

In [174]:
import re
help(re.split)

Help on function split in module re:

split(pattern, string, maxsplit=0, flags=0)
    Split the source string by the occurrences of the pattern,
    returning a list containing the resulting substrings.  If
    capturing parentheses are used in pattern, then the text of all
    groups in the pattern are also returned as part of the resulting
    list.  If maxsplit is nonzero, at most maxsplit splits occur,
    and the remainder of the string is returned as the final element
    of the list.



# get list of all skills in a list

In [194]:
flat_list = []
#print(type(l))
for sublist in l:
    #print(sublist)
    #print(type(sublist))
    item_list=re.split(',',sublist)
    #print(item_list)
    flat_list.extend(item_list)
    #for item in sublist:
        #flat_list.append(item)
#print(flat_list)

In [195]:
def rem_space(skill):
    return skill.strip()

# remove leading and trailing spaces from all elements of the list

In [198]:
flat_list_Series = pd.Series(flat_list) 
flat_list_Series=flat_list_Series.apply(rem_space)

In [202]:
total_skills=len(flat_list_Series.unique())

In [ ]:
count_skills_possesed=